In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas
import datetime
from datetime import datetime
import requests
import re
from selenium import webdriver
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


In [6]:
browser = webdriver.Chrome(executable_path='/Users/hugoruizverastegui/Downloads/chromedriver-131')

In [3]:
# automated install
#browser = webdriver.Chrome(ChromeDriverManager().install())

In [4]:


parameters= [
{
"country": "TH",   
"website": "OIS",   
"url_scrap": (("france","https://www.i-d-s.com/dubai/wine/wines-by-country/france.html?p="),    
("argentina","https://www.i-d-s.com/dubai/wine/wines-by-country/argentina.html?p="),
("australia","https://www.i-d-s.com/dubai/wine/wines-by-country/australia.html?p="),
("chile","https://www.i-d-s.com/dubai/wine/wines-by-country/chile.html?p="),
("new_zealand","https://www.i-d-s.com/dubai/wine/wines-by-country/new-zealand.html?p="),
("south_africa","https://www.i-d-s.com/dubai/wine/wines-by-country/south-africa.html?p="),
("uk","https://www.i-d-s.com/dubai/wine/wines-by-country/united-kingdom.html?p="),
("usa","https://www.i-d-s.com/dubai/wine/wines-by-country/usa.html?p="),
("portugal","https://www.i-d-s.com/dubai/wine/wines-by-country/portugal.html?p="),
("canada","https://www.i-d-s.com/dubai/wine/wines-by-country/canada.html?p="),
("united_kingdom","https://www.i-d-s.com/dubai/wine/wines-by-country/united-kingdom.html?p="),
("germany","https://www.i-d-s.com/dubai/wine/wines-by-country/germany.html?p="),
("italy","https://www.i-d-s.com/dubai/wine/wines-by-country/italy.html?p="),
("spain","https://www.i-d-s.com/dubai/wine/wines-by-country/spain.html?p="),
("rest-of-the-world","https://www.i-d-s.com/dubai/wine/wines-by-country/rest-of-the-world.html?p="),
("scotch_whisky","https://www.i-d-s.com/dubai/spirits/scotch-whisky.html?p="),
("other_whisky","https://www.i-d-s.com/dubai/spirits/bourbon-other-whiskey.html?p=")

),
    
    
"price_regex":  "finalPrice\" class=\"price-wrapper \"><span class=\"price\">.*?\$(.*?)<",    
#"price_regex":  "<span class=\"price\">\$(.*?)<",
##"data-price-amount=\"(.*?)\"",
"name_regex": "class=\"product-item-link\" href=.*?>(.*?)<",                
"url_regex": "product-item-link\" href=\"(.*?)\"",
"brand_regex": "NA", 
"scrolling_type" : "no scroll",
"pop_up_type" : "have pop up", 
"page_not_found_type" : "yes",
"page_not_found_regex":"find products matching the selection",
"format_change_type" : "yes",
"multiple_pages" : "yes",
"scrapper_activated":"1"
}
    
]

DF_final=pandas.DataFrame()

In [18]:
def pop_up_closing(param,browser_param):
    if param["pop_up_type"]=='no pop up':
        pass
    if param["pop_up_type"]=='have pop up':

        close=browser_param.find_elements_by_class_name("action-close")
        for element in close : 
            try:
                element.click()   
                time.sleep(1)
            except:
                pass

In [20]:
def get_data(param):
    
        l = len(param["url_scrap"])
        i=0
        df_aggregated=pandas.DataFrame()
        nb_issues = 0
        today_date=datetime.today().strftime('%Y-%m-%d')
        
        while i<l:
        #while i<2:
            
            #browser = webdriver.Chrome(executable_path=executable_path)
            #browser = webdriver.Chrome(ChromeDriverManager().install())
            browser=webdriver.Chrome(executable_path='/Users/hugoruizverastegui/Downloads/chromedriver-131')
            category=param["url_scrap"][i][0]           
            
            url=param["url_scrap"][i][1]
            print(category)
            #print(url)        

            # to change into a parameter
            test_not_found=param["page_not_found_regex"]

            #init
            page_not_found=0

            # different starting point depending on the param page not found 
            if param["page_not_found_type"]=='yes':
                page=1

            if param["page_not_found_type"]=='no':
                page=""


            while page_not_found==0:
                #use of try to handle possible scrapping errors or time outs - https://stackoverflow.com/questions/51943694/execute-when-error-occurs-python
                     
                url_with_page=url+str(page)
                print(url_with_page)     

                browser.get(url_with_page)    

                time.sleep(5)
                #use of the anti pop up function
                pop_up_closing(param,browser)

                #use of the scrolling function
                #scrolling(param,browser)

                #use of format changing
                #format_change(param,browser,page)

                # extract of sourc code 
                time.sleep(2)
                htmlsource = browser.page_source
                

                #use of the commas replace code
                #htmlsource_replaced = replace_commas(param,htmlsource)
                #print(htmlsource)

                time.sleep(2)

                name = re.findall(param["name_regex"], htmlsource)
                print("name: " + str(len(name)))
                print(name)
                
                url_product = re.findall(param["url_regex"], htmlsource)
                print("url_product: " + str(len(url_product)))
                print(url_product)
                
                prices = re.findall(param["price_regex"], htmlsource)
                print("prices: " + str(len(prices)))
                prices = prices[0:len(name)]
                print("reduced prices: " + str(len(prices)))
                print(prices)
                

                
                df=[prices,name,url_product]
                df=pandas.DataFrame(df).transpose()
                df['country']=category
                df_aggregated=df_aggregated.append(df)


                # exit loop after first iteration if there are no pages to go through
                if param["page_not_found_type"]=='no':
                    page_not_found = 1

                if param["page_not_found_type"]=='yes':
                    page_not_found = len(re.findall(test_not_found, htmlsource))
                    page=page+1
                    # only keep the output if there is no shift 


            i=i+1
            browser.close()
             
            print("---------------")
        
        #define columns names

        return df_aggregated





In [21]:
df_wine_diplo=get_data(parameters[0])

france
https://www.i-d-s.com/dubai/wine/wines-by-country/france.html?p=1
name: 12
['Nicolas Feuillatte Brut with Gift Box', 'Chateau Fleur de Pedesclaux', 'Domaine Cailbourdin Pouilly Fume ‘Les Racines’', '2018 Chateau Croix Mouton', '2010 Château Beychevelle, 4ème Cru Classé, Saint-Julien', '2020 Sarget D Gruaud Larose, Saint-Julien', '2017 Château Les Ormes de Pez Cru Bourgeois Saint Estephe', '2015 Château Corbin, Saint-Emilion', '2020 Brulieres de Beychevelle', "Clos l'Hermitage, Lalande de Pomerol", 'Château Tayac, Margaux', 'Château des Antonins, Bordeaux Supérieur']
url_product: 12
['https://www.i-d-s.com/dubai/nicolas-feuillatte-brut-with-gift-box-cp1066.html', 'https://www.i-d-s.com/dubai/chateau-fleur-de-pedesclaux-fr1399.html', 'https://www.i-d-s.com/dubai/domaine-cailbourdin-pouilly-fume-les-racines-fr1380.html', 'https://www.i-d-s.com/dubai/2018-chateau-croix-mouton-fw1253.html', 'https://www.i-d-s.com/dubai/2010-ch-teau-beychevelle-4-me-cru-class-saint-julien-fw1225.html'

/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_16448/1430148339.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


name: 12
['Château Mouras Graves, Bordeaux', 'Château Fleur de Lisse Saint-Émilion Grand Cru, Bordeaux', 'Baron Philippe de Rothschild, Mouton Cadet Blanc', 'Baron Philippe de Rothschild, Mouton Cadet Rouge', 'Taittinger Rose including 2 glasses', 'Domaine de Grange Neuve, Monbazillac', 'Louis Tete, Brouilly', 'Minuty Prestige Magnum (1.5L)', 'Maison Champy Volnay- BIO', '2020 Château Pibran, Pauillac', 'Domaine du Closel Clos du Papillon Savennières, 2020', 'Domaine Marcel Couturier  "Aux Scellés" Mâcon Blanc, 2020']
url_product: 12
['https://www.i-d-s.com/dubai/ch-teau-mouras-graves-bordeaux-fr1337.html', 'https://www.i-d-s.com/dubai/ch-teau-fleur-de-lisse-saint-milion-grand-cru-bordeaux-fr1336.html', 'https://www.i-d-s.com/dubai/baron-philippe-de-rothschild-mini-mouton-cadet-blanc-mw1000.html', 'https://www.i-d-s.com/dubai/baron-philippe-de-rothschild-mouton-cadet-rouge-mw1001.html', 'https://www.i-d-s.com/dubai/taittinger-rose-including-2-glasses-zz11289.html', 'https://www.i-d-s.c

/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_16448/1430148339.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


name: 12
['2019 Château Pape Clément Grand Cru Classé de Graves, Pessac-Léognan', 'Antoine Geoffroy Puligny Montrachet Vieilles Vignes', "Domaine Figeat L'Héritage` Pouilly-Fumé", 'Château Tour de Capet Saint-Émilion Grand Cru, Bordeaux', "Château Patache d'Aux Médoc Cru Bourgeois, Bordeaux", 'Domaine Chanson Corton Vergennes Grand Gru ', 'Chateau Lamarsalle Montagne Saint-Emilion', '2020  Château Beychevelle "Amiral De Beychevelle" Saint-Julien', 'La Chablisienne Chablis Grand Cru Les Clos', 'Pouilly Fume Les Cris Domaine Cailbourdin', 'Cristal Roederer Brut 2009 Magnum', 'Bollinger Special Cuvée Brut']
url_product: 12
['https://www.i-d-s.com/dubai/2019-ch-teau-pape-cl-ment-grand-cru-class-de-graves-pessac-l-ognan-fw1204.html', 'https://www.i-d-s.com/dubai/antoine-geoffroy-puligny-montrachet-vieilles-vignes-fr1303.html', 'https://www.i-d-s.com/dubai/domaine-figeat-l-h-ritage-pouilly-fum-fr1339.html', 'https://www.i-d-s.com/dubai/ch-teau-tour-de-capet-saint-milion-grand-cru-bordeaux-fr

...

/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_16448/1430148339.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


name: 5
['Knob Creek Small Batch, Kentucky Straight Bourbon', 'Woodford Reserve Double Oaked, Kentucky Straight Bourbon', 'Wild Turkey 8 Year Old 101 Proof, Kentucky Straight Bourbon Whiskey', 'Canadian Club', 'Crown Royal, Deluxe Whisky, in Gift Box']
url_product: 5
['https://www.i-d-s.com/dubai/knob-creek-small-batch-kentucky-straight-bourbon-wh2062.html', 'https://www.i-d-s.com/dubai/woodford-reserve-double-oaked-kentucky-straight-bourbon-wh1061.html', 'https://www.i-d-s.com/dubai/wild-turkey-8-year-old-101-proof-kentucky-straight-bourbon-whiskey-wh1088.html', 'https://www.i-d-s.com/dubai/canadian-club-wh1078.html', 'https://www.i-d-s.com/dubai/crown-royal-deluxe-whisky-in-gift-box-wh1079.html']
prices: 5
reduced prices: 5
['59.00', '98.60', '32.15', '19.10', '34.85']
https://www.i-d-s.com/dubai/spirits/bourbon-other-whiskey.html?p=6


/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_16448/1430148339.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


name: 0
[]
url_product: 0
[]
prices: 0
reduced prices: 0
[]
---------------


/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_16448/1430148339.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_aggregated=df_aggregated.append(df)


In [23]:
df_wine_diplo

,0,1,2,country
0,39.99,Nicolas Feuillatte Brut with Gift Box,https://www.i-d-s.com/dubai/nicolas-feuillatte...,france
1,37.00,Chateau Fleur de Pedesclaux,https://www.i-d-s.com/dubai/chateau-fleur-de-p...,france
2,34.30,Domaine Cailbourdin Pouilly Fume ‘Les Racines’,https://www.i-d-s.com/dubai/domaine-cailbourdi...,france
3,111.70,2018 Chateau Croix Mouton,https://www.i-d-s.com/dubai/2018-chateau-croix...,france
4,"1,487.50","2010 Château Beychevelle, 4ème Cru Classé, Sai...",https://www.i-d-s.com/dubai/2010-ch-teau-beych...,france
...,...,...,...,...
0,59.00,"Knob Creek Small Batch, Kentucky Straight Bourbon",https://www.i-d-s.com/dubai/knob-creek-small-b...,other_whisky
1,98.60,"Woodford Reserve Double Oaked, Kentucky Straig...",https://www.i-d-s.com/dubai/woodford-reserve-d...,other_whisky
2,32.15,"Wild Turkey 8 Year Old 101 Proof, Kentucky Str...",https://www.i-d-s.com/dubai/wild-turkey-8-year...,other_whisky
3,19.10,Canadian Club,https://www.i-d-s.com/dubai/canadian-club-wh10...,other_whisky


In [24]:
df_wine_diplo.to_csv('wine_14_01_25.csv')

In [25]:
df_wine_diplo['vivino_url']=''
df_wine_diplo['vivino_rating']=''

In [26]:
df_wine_diplo_backup=df_wine_diplo

In [27]:
#param for vivino scrapper
features_vivino=['url','rating']

rating_regexp="\"ratingValue\":(.*?),"
url_regexp='\"Product\",\"\@id\":\"(.*?)\",'
name_regexp="description\":\"Check out this (.*?) on Vivino" 
price_regexp="class=\"wine-price-value\">(.*?)</span>"
region_regexp="wine-regions\/(.*?)\""
country_regexp="wine-countries\/(.*?)\""
nb_ratings_regexp="(\d+) ratings"

In [28]:
def scrap_feature_vivino(feature_name,feature_regex,vivino_feature_list):
        if len(re.findall(feature_regex, htmlsource))==0:
            vivino_feature_inter_list=['-']
        else:
            vivino_feature_inter_list=[re.findall(url_regexp, htmlsource)[0]]
            
        vivino_feature_list=vivino_feature_list+vivino_feature_inter_list
    

# RUN 1

In [ ]:
# we first want to only scrape the wine from vivino - the whisky will be scraped separately
import pandas as pd 
df_wine_diplo=pd.read_csv('wine_14_01_25.csv')
#keep only the rows where [2] does not contain whisky

df_wine_diplo_vivino=df_wine_diplo[~df_wine_diplo['country'].str.contains('whisky', case=False, na=False)]



KeyError: 1

In [17]:
# Code to send a wine name to Vivino and scrape the first result


vivino_url_wine=[]
vivino_rating_wine=[]
vivino_name_wine=[]
vivino_price_wine=[]
vivino_country_wine=[]
vivino_region_wine=[]
vivino_nb_ratings_wine=[]

list_wine_search=df_wine_diplo_vivino['1']

# open browser
#browser = webdriver.Chrome(ChromeDriverManager().install())
browser=webdriver.Chrome(executable_path='/Users/hugoruizverastegui/Downloads/chromedriver-131')
browser.get('https://www.vivino.com/FR/en/')
#for each wine name in the list 
for search_txt in list_wine_search:
    # write the wine name
    #browser.get('https://www.vivino.com/FR/en/')  
    time.sleep(1)
    try:
        # find the input bar
        element=browser.find_element_by_class_name("searchBar_searchInput__Jje-K")
        element.clear()
        element.send_keys(search_txt)
        element.send_keys(Keys.RETURN)

        # get the first result
        time.sleep(2)

        htmlsource = browser.page_source
        rating_regexp="\"ratingValue\":(.*?),"
        url_regexp='\"Product\",\"\@id\":\"(.*?)\",'
        name_regexp="description\":\"Check out this (.*?) on Vivino" 
        price_regexp="class=\"wine-price-value\">(.*?)</span>"
        region_regexp="wine-regions\/(.*?)\""
        country_regexp="wine-countries\/(.*?)\""
        nb_ratings_regexp="(\d+) ratings"

        time.sleep(1)

        if len(re.findall(url_regexp, htmlsource))==0:
            vivino_url_wine_inter=['-']
        else:
            vivino_url_wine_inter=[re.findall(url_regexp, htmlsource)[0]]
        
        if len(re.findall(rating_regexp, htmlsource))==0:
            vivino_rating_wine_inter= ['-']
        else:
            vivino_rating_wine_inter= [re.findall(rating_regexp, htmlsource)[0]]
        
        if len(re.findall(price_regexp, htmlsource))==0:
            vivino_price_wine_inter= ['-']
        else:
            vivino_price_wine_inter= [re.findall(price_regexp, htmlsource)[0]]
               
        if len(re.findall(name_regexp, htmlsource))==0:
            vivino_name_wine_inter= ['-']
        else:
            vivino_name_wine_inter= [re.findall(name_regexp, htmlsource)[0]]
            
        if len(re.findall(country_regexp, htmlsource))==0:
            vivino_country_wine_inter= ['-']
        else:
            vivino_country_wine_inter= [re.findall(country_regexp, htmlsource)[0]]
        
        
        if len(re.findall(region_regexp, htmlsource))==0:
            vivino_region_wine_inter= ['-']
        else:
            vivino_region_wine_inter= [re.findall(region_regexp, htmlsource)[0]]
        
        if len(re.findall(nb_ratings_regexp, htmlsource))==0:
            vivino_nb_ratings_wine_inter= ['-']
        else:
            vivino_nb_ratings_wine_inter= [re.findall(nb_ratings_regexp, htmlsource)[0]]
        

        vivino_url_wine=vivino_url_wine+vivino_url_wine_inter
        vivino_rating_wine=vivino_rating_wine+vivino_rating_wine_inter
        vivino_name_wine=vivino_name_wine+vivino_name_wine_inter
        vivino_price_wine=vivino_price_wine+vivino_price_wine_inter
        vivino_country_wine=vivino_country_wine+vivino_country_wine_inter
        vivino_region_wine=vivino_region_wine+vivino_region_wine_inter
        vivino_nb_ratings_wine=vivino_nb_ratings_wine+vivino_nb_ratings_wine_inter
        
        time.sleep(1)

    except:
        # so the same thing but regenerate the browser and wait 60s
        print('blocked - waiting 60')
        browser.close()
        time.sleep(60)
        
        #browser = webdriver.Chrome(ChromeDriverManager().install())
        browser=webdriver.Chrome(executable_path='/Users/hugoruizverastegui/Downloads/chromedriver-131')
        browser.get('https://www.vivino.com/FR/en/')
        
        element=browser.find_element_by_class_name("searchBar_searchInput__Jje-K")
        element.clear()
        element.send_keys(search_txt)
        element.send_keys(Keys.RETURN)

        # get the first result
        time.sleep(2)

        htmlsource = browser.page_source
        rating_regexp="\"ratingValue\":(.*?),"
        url_regexp='\"Product\",\"\@id\":\"(.*?)\",'
        name_regexp="description\":\"Check out this (.*?) on Vivino" 
        price_regexp="class=\"wine-price-value\">(.*?)<\/span>"
        region_regexp="wine-regions\/(.*?)\""
        country_regexp="wine-countries\/(.*?)\""
        nb_ratings_regexp="(\d+) ratings"

        time.sleep(1)

        if len(re.findall(url_regexp, htmlsource))==0:
            vivino_url_wine_inter=['-']
        else:
            vivino_url_wine_inter=[re.findall(url_regexp, htmlsource)[0]]
        
        if len(re.findall(rating_regexp, htmlsource))==0:
            vivino_rating_wine_inter= ['-']
        else:
            vivino_rating_wine_inter= [re.findall(rating_regexp, htmlsource)[0]]
        
        if len(re.findall(price_regexp, htmlsource))==0:
            vivino_price_wine_inter= ['-']
        else:
            vivino_price_wine_inter= [re.findall(price_regexp, htmlsource)[0]]
               
        if len(re.findall(name_regexp, htmlsource))==0:
            vivino_name_wine_inter= ['-']
        else:
            vivino_name_wine_inter= [re.findall(name_regexp, htmlsource)[0]]
            
        if len(re.findall(country_regexp, htmlsource))==0:
            vivino_country_wine_inter= ['-']
        else:
            vivino_country_wine_inter= [re.findall(country_regexp, htmlsource)[0]]
        
        
        if len(re.findall(region_regexp, htmlsource))==0:
            vivino_region_wine_inter= ['-']
        else:
            vivino_region_wine_inter= [re.findall(region_regexp, htmlsource)[0]]
        
        if len(re.findall(nb_ratings_regexp, htmlsource))==0:
            vivino_nb_ratings_wine_inter= ['-']
        else:
            vivino_nb_ratings_wine_inter= [re.findall(nb_ratings_regexp, htmlsource)[0]]
        

        vivino_url_wine=vivino_url_wine+vivino_url_wine_inter
        vivino_rating_wine=vivino_rating_wine+vivino_rating_wine_inter
        vivino_name_wine=vivino_name_wine+vivino_name_wine_inter
        vivino_price_wine=vivino_price_wine+vivino_price_wine_inter
        vivino_country_wine=vivino_country_wine+vivino_country_wine_inter
        vivino_region_wine=vivino_region_wine+vivino_region_wine_inter
        vivino_nb_ratings_wine=vivino_nb_ratings_wine+vivino_nb_ratings_wine_inter
        
    

In [18]:
already_scrapped_len=len(vivino_name_wine)
already_scrapped_len

767

In [19]:


remaining_list_wine=list_wine_search[already_scrapped_len:]
remaining_list_wine

Series([], Name: 1, dtype: object)

# Step 2 - redo for wines that did not get scrapped


In [39]:
browser=webdriver.Chrome(executable_path='/Users/hugoruizverastegui/Downloads/chromedriver-121')
browser.get('https://www.vivino.com/FR/en/')
#for each wine name in the list 
for search_txt in remaining_list_wine:
    # write the wine name
    #browser.get('https://www.vivino.com/FR/en/')  
    time.sleep(1)
    try:
        # find the input bar
        element=browser.find_element_by_class_name("searchBar_searchInput__Jje-K")
        element.clear()
        element.send_keys(search_txt)
        element.send_keys(Keys.RETURN)

        # get the first result
        time.sleep(2)

        htmlsource = browser.page_source
        rating_regexp="\"ratingValue\":(.*?),"
        url_regexp='\"Product\",\"\@id\":\"(.*?)\",'
        name_regexp="description\":\"Check out this (.*?) on Vivino" 
        price_regexp="class=\"wine-price-value\">(.*?)</span>"
        region_regexp="wine-regions\/(.*?)\""
        country_regexp="wine-countries\/(.*?)\""
        nb_ratings_regexp="(\d+) ratings"

        time.sleep(1)

        if len(re.findall(url_regexp, htmlsource))==0:
            vivino_url_wine_inter=['-']
        else:
            vivino_url_wine_inter=[re.findall(url_regexp, htmlsource)[0]]
        
        if len(re.findall(rating_regexp, htmlsource))==0:
            vivino_rating_wine_inter= ['-']
        else:
            vivino_rating_wine_inter= [re.findall(rating_regexp, htmlsource)[0]]
        
        if len(re.findall(price_regexp, htmlsource))==0:
            vivino_price_wine_inter= ['-']
        else:
            vivino_price_wine_inter= [re.findall(price_regexp, htmlsource)[0]]
               
        if len(re.findall(name_regexp, htmlsource))==0:
            vivino_name_wine_inter= ['-']
        else:
            vivino_name_wine_inter= [re.findall(name_regexp, htmlsource)[0]]
            
        if len(re.findall(country_regexp, htmlsource))==0:
            vivino_country_wine_inter= ['-']
        else:
            vivino_country_wine_inter= [re.findall(country_regexp, htmlsource)[0]]
        
        
        if len(re.findall(region_regexp, htmlsource))==0:
            vivino_region_wine_inter= ['-']
        else:
            vivino_region_wine_inter= [re.findall(region_regexp, htmlsource)[0]]
        
        if len(re.findall(nb_ratings_regexp, htmlsource))==0:
            vivino_nb_ratings_wine_inter= ['-']
        else:
            vivino_nb_ratings_wine_inter= [re.findall(nb_ratings_regexp, htmlsource)[0]]
        

        vivino_url_wine=vivino_url_wine+vivino_url_wine_inter
        vivino_rating_wine=vivino_rating_wine+vivino_rating_wine_inter
        vivino_name_wine=vivino_name_wine+vivino_name_wine_inter
        vivino_price_wine=vivino_price_wine+vivino_price_wine_inter
        vivino_country_wine=vivino_country_wine+vivino_country_wine_inter
        vivino_region_wine=vivino_region_wine+vivino_region_wine_inter
        vivino_nb_ratings_wine=vivino_nb_ratings_wine+vivino_nb_ratings_wine_inter
        
        time.sleep(1)

    except:
        # so the same thing but regenerate the browser and wait 60s
        print('blocked - waiting 60')
        browser.close()
        time.sleep(60)
        
        #browser = webdriver.Chrome(ChromeDriverManager().install())
        browser=webdriver.Chrome(executable_path='/Users/hugoruizverastegui/Downloads/chromedriver-121')
        browser.get('https://www.vivino.com/FR/en/')
        
        element=browser.find_element_by_class_name("searchBar_searchInput__Jje-K")
        element.clear()
        element.send_keys(search_txt)
        element.send_keys(Keys.RETURN)

        # get the first result
        time.sleep(2)

        htmlsource = browser.page_source
        rating_regexp="\"ratingValue\":(.*?),"
        url_regexp='\"Product\",\"\@id\":\"(.*?)\",'
        name_regexp="description\":\"Check out this (.*?) on Vivino" 
        price_regexp="class=\"wine-price-value\">(.*?)<\/span>"
        region_regexp="wine-regions\/(.*?)\""
        country_regexp="wine-countries\/(.*?)\""
        nb_ratings_regexp="(\d+) ratings"

        time.sleep(1)

        if len(re.findall(url_regexp, htmlsource))==0:
            vivino_url_wine_inter=['-']
        else:
            vivino_url_wine_inter=[re.findall(url_regexp, htmlsource)[0]]
        
        if len(re.findall(rating_regexp, htmlsource))==0:
            vivino_rating_wine_inter= ['-']
        else:
            vivino_rating_wine_inter= [re.findall(rating_regexp, htmlsource)[0]]
        
        if len(re.findall(price_regexp, htmlsource))==0:
            vivino_price_wine_inter= ['-']
        else:
            vivino_price_wine_inter= [re.findall(price_regexp, htmlsource)[0]]
               
        if len(re.findall(name_regexp, htmlsource))==0:
            vivino_name_wine_inter= ['-']
        else:
            vivino_name_wine_inter= [re.findall(name_regexp, htmlsource)[0]]
            
        if len(re.findall(country_regexp, htmlsource))==0:
            vivino_country_wine_inter= ['-']
        else:
            vivino_country_wine_inter= [re.findall(country_regexp, htmlsource)[0]]
        
        
        if len(re.findall(region_regexp, htmlsource))==0:
            vivino_region_wine_inter= ['-']
        else:
            vivino_region_wine_inter= [re.findall(region_regexp, htmlsource)[0]]
        
        if len(re.findall(nb_ratings_regexp, htmlsource))==0:
            vivino_nb_ratings_wine_inter= ['-']
        else:
            vivino_nb_ratings_wine_inter= [re.findall(nb_ratings_regexp, htmlsource)[0]]
        

        vivino_url_wine=vivino_url_wine+vivino_url_wine_inter
        vivino_rating_wine=vivino_rating_wine+vivino_rating_wine_inter
        vivino_name_wine=vivino_name_wine+vivino_name_wine_inter
        vivino_price_wine=vivino_price_wine+vivino_price_wine_inter
        vivino_country_wine=vivino_country_wine+vivino_country_wine_inter
        vivino_region_wine=vivino_region_wine+vivino_region_wine_inter
        vivino_nb_ratings_wine=vivino_nb_ratings_wine+vivino_nb_ratings_wine_inter
        

In [20]:
len(vivino_url_wine)

767

In [21]:
df_wine_diplo_vivino['vivino_url']=vivino_url_wine
df_wine_diplo_vivino['vivino_rating']=vivino_rating_wine
df_wine_diplo_vivino['vivino_name']=vivino_name_wine
df_wine_diplo_vivino['vivino_price']=vivino_price_wine
df_wine_diplo_vivino['vivino_country']=vivino_country_wine
df_wine_diplo_vivino['vivino_region']=vivino_region_wine
df_wine_diplo_vivino['nb_ratings']=vivino_nb_ratings_wine

/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_20101/2867374674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wine_diplo_vivino['vivino_url']=vivino_url_wine
/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_20101/2867374674.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wine_diplo_vivino['vivino_rating']=vivino_rating_wine
/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_20101/2867374674.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [22]:
df_wine_diplo_vivino

,Unnamed: 0,0,1,2,country,vivino_url,vivino_rating,vivino_name,vivino_price,vivino_country,vivino_region,nb_ratings
0,0,39.99,Nicolas Feuillatte Brut with Gift Box,https://www.i-d-s.com/dubai/nicolas-feuillatte...,france,https://www.vivino.com/FR/en/wines/157213587,3.9,Nicolas Feuillatte Réserve Exclusive Brut Cham...,—,france,champagne-grand-cru-chouilly,26179
1,1,37.00,Chateau Fleur de Pedesclaux,https://www.i-d-s.com/dubai/chateau-fleur-de-p...,france,https://www.vivino.com/FR/en/wines/1931635,4.0,Château Pédesclaux Fleur de Pédesclaux Pauillac 0,—,france,pauillac,7166
2,2,34.30,Domaine Cailbourdin Pouilly Fume ‘Les Racines’,https://www.i-d-s.com/dubai/domaine-cailbourdi...,france,https://www.vivino.com/FR/en/wines/173651106,4.0,Domaine A Cailbourdin Pouilly Fume Les Racines 0,—,france,pouilly-fume,223
3,3,111.70,2018 Chateau Croix Mouton,https://www.i-d-s.com/dubai/2018-chateau-croix...,france,https://www.vivino.com/FR/en/wines/4733991,3.6,Château Croix Mouton Bordeaux Supérieur 0,—,france,bordeaux-superieur,10451
4,4,"1,487.50","2010 Château Beychevelle, 4ème Cru Classé, Sai...",https://www.i-d-s.com/dubai/2010-ch-teau-beych...,france,https://www.vivino.com/FR/en/wines/2048358,4.3,Château Beychevelle Saint-Julien (Grand Cru Cl...,—,france,saint-julien,29432
...,...,...,...,...,...,...,...,...,...,...,...,...
762,4,29.10,"Grace Koshu Kayagatake, Grace Winery, Yamanashi",https://www.i-d-s.com/dubai/grace-koshu-kayaga...,rest-of-the-world,https://www.vivino.com/FR/en/wines/3542812,3.6,Grace Gris de Koshu 0,—,japan,yamanashi-ken,2839
763,5,35.40,"Grace Koshu Private Reserve, Grace Winery Grac...",https://www.i-d-s.com/dubai/grace-koshu-privat...,rest-of-the-world,https://www.vivino.com/FR/en/wines/4910969,3.7,Grace Toriibira Vineyard Koshu Private Reserve 0,—,japan,yamanashi-ken,874
764,6,34.90,"Grace Koshu Hishiyama, Private Reserve, Grace ...",https://www.i-d-s.com/dubai/grace-koshu-hishiy...,rest-of-the-world,https://www.vivino.com/FR/en/wines/167488945,3.7,Grace Hishiyama Vineyard Private Reserve Koshu 0,39.00,japan,yamanashi-ken,60
765,7,9.00,Calusari Pinot Grigio,https://www.i-d-s.com/dubai/calusari-pinot-gri...,rest-of-the-world,https://www.vivino.com/FR/en/wines/3130030,3.7,Călușari Pinot Grigio 0,—,romania,recas,2295


In [23]:
#check name distance 
import fuzzywuzzy
from fuzzywuzzy import fuzz

def distance_name(name1, name2):
    return fuzz.ratio(name1.lower(),name2.lower())

In [25]:
df_wine_diplo_vivino['ratio']=df_wine_diplo_vivino.apply(lambda x: distance_name(x[1], x['vivino_name']), axis=1)

/var/folders/9z/sjrbpv9j1996t3_4wv3ry65c0000gn/T/ipykernel_20101/1393062776.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wine_diplo_vivino['ratio']=df_wine_diplo_vivino.apply(lambda x: distance_name(x[1], x['vivino_name']), axis=1)


In [26]:
df_wine_diplo_vivino[df_wine_diplo_vivino['ratio']<=50]

,Unnamed: 0,0,1,2,country,vivino_url,vivino_rating,vivino_name,vivino_price,vivino_country,vivino_region,nb_ratings,ratio
0,0,39.99,Nicolas Feuillatte Brut with Gift Box,https://www.i-d-s.com/dubai/nicolas-feuillatte...,france,https://www.vivino.com/FR/en/wines/157213587,3.9,Nicolas Feuillatte Réserve Exclusive Brut Cham...,—,france,champagne-grand-cru-chouilly,26179,0
1,1,37.00,Chateau Fleur de Pedesclaux,https://www.i-d-s.com/dubai/chateau-fleur-de-p...,france,https://www.vivino.com/FR/en/wines/1931635,4.0,Château Pédesclaux Fleur de Pédesclaux Pauillac 0,—,france,pauillac,7166,4
2,2,34.30,Domaine Cailbourdin Pouilly Fume ‘Les Racines’,https://www.i-d-s.com/dubai/domaine-cailbourdi...,france,https://www.vivino.com/FR/en/wines/173651106,4.0,Domaine A Cailbourdin Pouilly Fume Les Racines 0,—,france,pouilly-fume,223,4
3,3,111.70,2018 Chateau Croix Mouton,https://www.i-d-s.com/dubai/2018-chateau-croix...,france,https://www.vivino.com/FR/en/wines/4733991,3.6,Château Croix Mouton Bordeaux Supérieur 0,—,france,bordeaux-superieur,10451,4
4,4,"1,487.50","2010 Château Beychevelle, 4ème Cru Classé, Sai...",https://www.i-d-s.com/dubai/2010-ch-teau-beych...,france,https://www.vivino.com/FR/en/wines/2048358,4.3,Château Beychevelle Saint-Julien (Grand Cru Cl...,—,france,saint-julien,29432,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,4,29.10,"Grace Koshu Kayagatake, Grace Winery, Yamanashi",https://www.i-d-s.com/dubai/grace-koshu-kayaga...,rest-of-the-world,https://www.vivino.com/FR/en/wines/3542812,3.6,Grace Gris de Koshu 0,—,japan,yamanashi-ken,2839,8
763,5,35.40,"Grace Koshu Private Reserve, Grace Winery Grac...",https://www.i-d-s.com/dubai/grace-koshu-privat...,rest-of-the-world,https://www.vivino.com/FR/en/wines/4910969,3.7,Grace Toriibira Vineyard Koshu Private Reserve 0,—,japan,yamanashi-ken,874,4
764,6,34.90,"Grace Koshu Hishiyama, Private Reserve, Grace ...",https://www.i-d-s.com/dubai/grace-koshu-hishiy...,rest-of-the-world,https://www.vivino.com/FR/en/wines/167488945,3.7,Grace Hishiyama Vineyard Private Reserve Koshu 0,39.00,japan,yamanashi-ken,60,4
765,7,9.00,Calusari Pinot Grigio,https://www.i-d-s.com/dubai/calusari-pinot-gri...,rest-of-the-world,https://www.vivino.com/FR/en/wines/3130030,3.7,Călușari Pinot Grigio 0,—,romania,recas,2295,7


In [ ]:
# for the wine with ratio <50 they are scrapped a second time

In [28]:
df_wine_diplo_vivino.to_csv('wine_diplo_2025_14Jan.csv')